In [1]:
# Import numpy
import numpy as np

# Import & Init jieba
import jieba
jieba.set_dictionary('datas/dict/dict.txt.big')
jieba.load_userdict('datas/dict/edu_dict.txt')

# Import pandas
import pandas as pd
from pandas import Series, DataFrame

# Import util
import time
import re
import sys
import gc
import json

Building prefix dict from /Users/sunset/Talk2AI_Contest/datas/dict/dict.txt.big ...
Loading model from cache /var/folders/43/l4vp_w_x4wb11mmy_bb1jrkc0000gn/T/jieba.u857f67a870683287981bc6f5b9493ffc.cache
Loading model cost 1.907 seconds.
Prefix dict has been built succesfully.


### Load datasets

#### sample test data

In [2]:
sample = pd.read_csv('datas/sample_test_data.txt')
sample

,id,dialogue,options,answer
0,0,A:你這麼快就知道了,B:全家就是你家\tB:付出不是浪費時間\tB:願意為社會付出的人太少了\tB:我都是一個人...,4
1,1,A:每一支冰塊都不同 水質不同 硬度不同,A:紋路不同\tA:也是你唯一發洩情緒的辦法吧\tA:還記不記得那次你在我家巷口\tA:像拼...,0
2,2,A:這樣沖這麼一大塊網子 要多久時間,B:有更舒適的環境\tB:感覺是一隻很貼心的貓咪\tB:我相信一定可以成功的啦\tB:可以講...,5
3,3,A:兒子啊 都幾點了 你還不睡,B:沒關係啦 我來幫你\tB:我在背書啦 就快背好了\tB:造成誤會 不是很可惜嗎\tB:女...,1
4,4,A:孩子還在念書的時候 看到其他小朋友都有父母 而自己沒有媽媽,A:那孩子就是突然\tA:就是說\tA:禮拜一到學校\tA:媽媽還不會開車\tA:就會問我媽...,4
5,5,A:那牠吐絲這三天三夜\tA:你們不就都要一直在這邊守護著牠囉,B:涂先生\tB:那這些蠶寶寶在這邊吐絲\tB:要吐多久啊\tB:溫度三十多度的話\tB:對...,4
6,6,A:奇怪了 濂僑跑到哪兒去了呢 濂僑 你怎麼啦,B:我可能吃壞肚子了 肚子好痛喔\tB:另外一種比法是 一星期有七天\tB:為什麼不可以 同...,0
7,7,A:是什麼讓玉屏的改變這麼大啊,B:當然是紀老師的魔法囉\tB:都交給這位大力士吧\tB:粗重的交給我就對了\tB:我是八年...,0
8,8,A:曉書啊 你手上拿的 該不是最新的遊戲機吧,B:你不要小看這個扯鈴\t B:互相幫忙一下嘛\tB:一個週末出去走走 沒有關係的啦\tB:...,3
9,9,A:失戀的確會讓人家很難過,A:所以你要縮短認真難過的時間\tA:他走了十三個年頭\tA:獲選之後當然是很開心囉\tA:...,0


In [3]:
# Extract sample test datas
x1 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in sample.dialogue.values]
x2 = [[s for s in re.sub('[A-Z]:', '\t', _).split('\t') if len(s.strip())] for _ in sample.options.values]

# Tokenize
x1 = np.array([list(jieba.cut(' '.join(_))) for _ in x1])
x2 = np.array([[list(jieba.cut(s)) for s in _] for _ in x2])
y = np.array(sample.answer.values)
assert(np.sum([len(_)!=6 for _ in x2]) == 0)

### Counting PPT Gossiping QA

In [4]:
# # Extract PPT Gossiping QA datas
# with open('datas/raw/Gossiping-QA-Dataset.txt', 'r') as fi:
#     gossip_lines = [line.strip().split('\t') for line in fi]
# gossip_x1 = [list(jieba.cut(line[0])) for line in gossip_lines if len(line) == 2]
# gossip_x2 = [list(jieba.cut(line[1])) for line in gossip_lines if len(line) == 2]

In [5]:
# gossip_x1 = [[w.strip() for w in seg if w.strip()!=''] for seg in gossip_x1]
# gossip_x2 = [[w.strip() for w in seg if w.strip()!=''] for seg in gossip_x2]

In [6]:
# cnt = {}
# next_cnt = {}
# for i, (q_seg, r_seg) in enumerate(zip(gossip_x1, gossip_x2)):
#     for qw in q_seg:
#         if qw not in cnt:
#             cnt[qw] = 0
#             next_cnt[qw] = {}
#         cnt[qw] += 1
#         now = next_cnt[qw]
#         for rw in r_seg:
#             if rw not in now:
#                 now[rw] = 0
#             now[rw] += 1

In [7]:
# with open('datas/dict/cnt.json', 'w') as fo:
#     json.dump(cnt, fo)
# with open('datas/dict/next_cnt.json', 'w') as fo:
#     json.dump(next_cnt, fo)

cnt = json.load(open('datas/dict/cnt.json'))
next_cnt = json.load(open('datas/dict/next_cnt.json'))

### Testing

In [56]:
def belief(w):
    return cnt[w] if w in cnt else 0

def cnt_score(w, nw, alpha=0):
    if w not in next_cnt or nw not in next_cnt[w]:
        return 0
    return next_cnt[w][nw] / np.log(cnt[w] + alpha)

def sim(s1, s2, alpha=0, beta=1):
    score = []
    p = beta
    for i in range(len(s1)):
        if s1[i] == ' ':
            p *= beta
            continue
        for j in range(len(s2)):
            score.append(cnt_score(s1[i], s2[j], alpha))
    return np.mean(score)

In [57]:
correct = 0
for q, rs, ans in zip(x1, x2, y):
    scores = [sim(q, r, 1) for r in rs]
    correct += int(np.argmax(scores) == ans)
    print(q)
    for i in range(6):
        print('%d %6.2f' % (i, scores[i]), rs[i])
    print(ans)
    print('=' * 100)
print('%4d / %4d' % (correct, len(y)))

['你', '這麼', '快', '就', '知道', '了']
0   5.85 ['全家', '就是', '你家']
1   5.60 ['付出', '不是', '浪費時間']
2  44.05 ['願意', '為', '社會', '付出', '的', '人太少', '了']
3  38.04 ['我', '都', '是', '一個', '人', '把', '他關', '在', '家裡']
4  15.06 ['廢話', ' ', '你', '沒有', '聽過', '壞話', '傳', '千里']
5  37.93 ['你', '不要', '以為', '你', '拳頭', '大', ' ', '大家', '都', '要服', '你', ' ', '我', '才', '不怕', '你']
4
['每', '一支', '冰塊', '都', '不同', ' ', '水質', '不同', ' ', '硬度', '不同']
0   0.42 ['紋路', '不同']
1  28.95 ['也是', '你', '唯一', '發洩', '情緒', '的', '辦法', '吧']
2  13.82 ['還記', '不', '記得', '那次', '你', '在', '我家', '巷口']
3  32.65 ['像', '拼命三郎', '一樣', '的', '跳舞']
4  17.60 ['雖然', '我', '不知道', '你', '為什麼', '這麼', '憤怒']
5  12.01 ['不過', '我', '真的', '受到', '感動', '喔']
0
['這樣', '沖', '這麼', '一大塊', '網子', ' ', '要', '多久', '時間']
0  11.41 ['有', '更', '舒適', '的', '環境']
1  10.32 ['感覺', '是', '一隻', '很', '貼心', '的', '貓咪']
2   9.61 ['我', '相信', '一定', '可以', '成功', '的', '啦']
3  10.00 ['可以', '講心', '裡面', '的', '悄悄話']
4   3.07 ['颱風', '那', '一天', '風雨', '有', '很大', '嗎']
5   1.81 ['差不多', '要', '兩個', '小時']
5
['

In [65]:
_ = [(v, k) for k, v in next_cnt['嘛'].items()]
_.sort(reverse=True)
_

[(170, '的'),
 (85, '，'),
 (73, '你'),
 (70, '是'),
 (66, '都'),
 (63, '我'),
 (60, '了'),
 (52, '有'),
 (50, '好'),
 (47, '人'),
 (46, '就'),
 (44, '在'),
 (42, '？'),
 (39, '嗎'),
 (36, '會'),
 (32, '台灣'),
 (29, '中國'),
 (28, '跟'),
 (27, '被'),
 (27, '去'),
 (26, '說'),
 (26, '要'),
 (25, '啊'),
 (23, '肥宅'),
 (23, '?'),
 (22, '很'),
 (21, '沒'),
 (21, '可以'),
 (21, '什麼'),
 (19, '這樣'),
 (18, '現在'),
 (18, '幹'),
 (18, '他'),
 (18, '...'),
 (17, '就是'),
 (17, '吧'),
 (17, '吃'),
 (17, '不是'),
 (17, '一堆'),
 (16, '自己'),
 (16, '看'),
 (15, '啦'),
 (15, '到'),
 (15, '也'),
 (15, '~'),
 (14, '覺得'),
 (14, '美國'),
 (14, '真的'),
 (14, '比較'),
 (14, '才'),
 (14, '問題'),
 (14, '不要'),
 (13, '！'),
 (13, '知道'),
 (12, '阿'),
 (12, '打'),
 (12, '她'),
 (12, '喔'),
 (12, '再'),
 (12, '不知道'),
 (12, '不會'),
 (11, '這'),
 (11, '應該'),
 (11, '想'),
 (11, '也是'),
 (11, '不'),
 (11, '一樣'),
 (10, '用'),
 (10, '屌'),
 (10, '叫'),
 (10, '先'),
 (10, ','),
 (9, '這種'),
 (9, '然後'),
 (9, '很多'),
 (9, '多'),
 (9, '做'),
 (9, '以為'),
 (9, '。'),
 (8, '還是'),
 (8, '還'),
 (8, 

In [55]:
for q, rs, ans in zip(x1, x2, y):
    if '為什麼' in q and '因為' in rs[ans]:
        print(q)
        print(rs[ans])
        print('=' * 100)
print('\n\n\n')
for q, rs, ans in zip(x1, x2, y):
    if '為什麼' in q and '因為' not in rs[ans]:
        print(q)
        print(rs[ans])
        print('=' * 100)

['那', '你', '知道', '為什麼', '會', '有', '攀岩', '場嗎', ' ', ' ', '為什麼']
['因為', '那邊', '的', '同學', '體育', '都', '非常', '好', ' ', '很會', '爬樹']
['亞婷', ' ', '你', '在', '刷', '馬', '嗎', ' ', '對', '啊', ' ', '為什麼', '要', '這樣', '刷馬', '呢']
['因為', '牠', '這樣', '會', '比較', '舒服']




['可是', '為什麼', '會', '選擇', '到', ' ', '歐洲', '的', '布拉格', '那個', '城市', ' ', '去', '學習', '戲劇', '呢']
['我', '覺得', '主要', '是', '它', '的', '環境']
['則緯', '對', '你', '這麼', '好', ' ', '為什麼', '要', '拒絕', '他', ' ', '因為', '我', '只', '想', ' ', '跟則緯', '繼續', '當好', '同學', '就', '好', '了']
['可是', '我', '看則緯', '想當', '的', ' ', '不', '只是', '好', '同學', '吧']


In [ ]:
for q, rs, ans in zip(x1, x2, y):
    if '為什麼' in q and '因為' in rs[ans]:
        print(q)
        print(rs[ans])
        print('=' * 100)